In [1]:
# Run in terminal!
# vllm serve "ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16"

In [2]:
import sys
print(sys.version)

3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:08:17) [GCC 12.2.0]


In [3]:
from openai import OpenAI
import os
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

2025-04-18 12:36:18.627281: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744994178.645157  128888 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744994178.650689  128888 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744994178.664521  128888 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744994178.664537  128888 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744994178.664539  128888 computation_placer.cc:177] computation placer alr

In [4]:
# Load BLIP-2
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Use the smaller BLIP-2 model with OPT
model_id = "Salesforce/blip2-opt-2.7b"
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

model.safetensors.index.json:   0%|          | 0.00/122k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [7]:
# Load image
image = Image.open("jeans.png").convert("RGB")

# Use instruction-style prompt as a QUESTION (not declarative)
prompt = "What is the person wearing? Describe color, fabric, and style."

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Process
#inputs = processor(image, prompt, return_tensors="pt").to("cpu")

# Generate
with torch.no_grad():
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        temperature=0.7
    )

# Decode
caption = processor.decode(generated_ids[0], skip_special_tokens=True)
print("🧵 Caption:", caption)

/home/arg8qqv/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🧵 Caption: What is the person wearing? Describe color, fabric, and style.



In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")  # force CPU

image = Image.open("jeans.png").convert("RGB")
prompt = ""  # <-- this is key

inputs = processor(image, prompt, return_tensors="pt").to("cpu")
outputs = model.generate(**inputs, max_length=50)
caption = processor.decode(outputs[0], skip_special_tokens=True)

print("Caption:", caption)

Caption: a woman in a white top and blue pants


In [9]:
# Light-weight BLIP model (needs ~2GB GPU)
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

model_id = "Salesforce/blip-image-captioning-base"  # or blip-image-captioning-large
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id).to("cpu")#to("cuda" if torch.cuda.is_available() else "cpu")

image = Image.open("jeans.png").convert("RGB")
prompt = "Describe the primary piece of clothing. Describe colors, patterns, fabrics, and styles."

inputs = processor(image, prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=50)
print(processor.decode(outputs[0], skip_special_tokens=True))

describe the primary piece of clothing. describe colors, patterns, fabrics, and styles.s


In [10]:
# Point to your local VLLM server
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="sk-fake-key"  # Any string works; vLLM doesn’t check this
)

In [11]:
message = f'Return just the description of the article of clothing: {caption}. Return no other text.'

response = client.chat.completions.create(
    model="ConfidentialMind/Mistral-Small-24B-Instruct-2501_GPTQ_G32_W4A16",
    messages=[
        {"role": "user", "content": message}
    ],
    temperature=0.15
)

# Print the response
print(response.choices[0].message.content)

APIConnectionError: Connection error.